In [1]:
import os 
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = 2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [23]:
dataset = keras.preprocessing.image_dataset_from_directory(
    directory='faces_6class/all', label_mode=None, image_size=(64, 64),
    batch_size = 8, shuffle=True
).map(lambda x: (x/255.0 - 0.5)*2)


discriminator = keras.Sequential(
    [
        keras.Input(shape=(64, 64, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid'),
    ]
)

print(discriminator.summary())


Found 614 files belonging to 1 classes.
Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 32, 32, 64)        3136      
                                                                 
 leaky_re_lu_61 (LeakyReLU)  (None, 32, 32, 64)        0         
                                                                 
 conv2d_22 (Conv2D)          (None, 16, 16, 128)       131200    
                                                                 
 leaky_re_lu_62 (LeakyReLU)  (None, 16, 16, 128)       0         
                                                                 
 conv2d_23 (Conv2D)          (None, 8, 8, 128)         262272    
                                                                 
 leaky_re_lu_63 (LeakyReLU)  (None, 8, 8, 128)         0         
                                                                 
 flatten_3 (F

In [24]:
latent_dim = 128

generator = keras.Sequential(
    [
        layers.Input(shape=(latent_dim,)),
        layers.Dense(8*8*128),
        layers.Reshape((8, 8, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Conv2D(3, kernel_size=5, padding='same', activation='tanh'),
    ]
)

print(generator.summary())

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 8192)              1056768   
                                                                 
 reshape_12 (Reshape)        (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose_52 (Conv2D  (None, 16, 16, 128)      262272    
 Transpose)                                                      
                                                                 
 leaky_re_lu_64 (LeakyReLU)  (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_53 (Conv2D  (None, 32, 32, 256)      524544    
 Transpose)                                                      
                                                                 
 leaky_re_lu_65 (LeakyReLU)  (None, 32, 32, 256)     

In [2]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping: no known devices.


In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [25]:
opt_gen = keras.optimizers.Adam(1e-4)
opt_disc = keras.optimizers.Adam(1e-4)
loss_fn = keras.losses.BinaryCrossentropy()

for epoch in range(500):
    for idx, real in enumerate(tqdm(dataset)):
        batch_size = real.shape[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
        fake = generator(random_latent_vectors)

        if idx % 100 == 0:
            img = keras.preprocessing.image.array_to_img(fake[0])
            img.save(f"generated/generated_img{epoch}_{idx}_.png")

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z))
        with tf.GradientTape() as disc_tape:
            loss_disc_real = loss_fn(tf.ones((batch_size, 1)), discriminator(real))
            loss_disc_fake = loss_fn(tf.zeros(batch_size, 1), discriminator(fake))
            loss_disc = (loss_disc_real + loss_disc_fake)/2

        grads = disc_tape.gradient(loss_disc, discriminator.trainable_weights)
        opt_disc.apply_gradients(
            zip(grads, discriminator.trainable_weights)
        )

        ### Train Generator min log(1 - D(G(z)) <-> max log(D(G(z))
        with tf.GradientTape() as gen_tape:
            fake = generator(random_latent_vectors)
            output = discriminator(fake)
            loss_gen = loss_fn(tf.ones(batch_size, 1), output)

        grads = gen_tape.gradient(loss_gen, generator.trainable_weights)
        opt_gen.apply_gradients(
            zip(grads, generator.trainable_weights)
        )



generator.save_weights('gen_w')
discriminator.save_weights('dis_w')



 86%|████████▌ | 66/77 [02:26<00:24,  2.22s/it]


KeyboardInterrupt: 